In [4]:
import cv2
import numpy as np
import os

from predict_numbers import showImage, saveImage, homography_board, create_bb

In [5]:
def get_hex_images(image, dst_points, side_length, num_offset):
    """ Get all the images for a NN to identify hex types
    dst_points: perimiter points of the board
    side_length: side length of bounding box for each number
    num_offset: the number of pixels above the number point
    """
    subimages = []
    R = 526
    # This is the actual center point
    x1, y1, x2, y2 = create_bb(image,
                               (R, R - num_offset),
                               side_length)
    print(type(x1), type(y1), type(x2), type(y2))
    subimage_center = image[y1:y2, x1:x2]
    subimages.append(subimage_center)
    # This gets all the numbers that fall on the line between the center point of the board and the perimiter points
    for pt in dst_points:
        delta_x = R - pt[0]
        delta_y = R - pt[1]

        x1, y1, x2, y2 = create_bb(image, 
                                   (int((R + (7/24)*delta_x)), 
                                    int((R + (7/24)*delta_y) - num_offset)), 
                                   side_length)
        subimage1 = image[y1:y2, x1:x2]

        x1, y1, x2, y2 = create_bb(image, 
                                   (int((R + (3/5)*delta_x)), 
                                    int((R + (3/5)*delta_y) - num_offset)), 
                                    side_length)
        subimage2 = image[y1:y2, x1:x2]

        subimages.append(subimage1)
        subimages.append(subimage2)

    # This gets all the numbers not found by the center lines to the perimiter points using every other perimiter point to find the numbers
    for i in range(len(dst_points)):
        # Draw lines between every other corner
        next_point_ind = i+2
        if i+2 >= len(dst_points):
            next_point_ind = i+2-len(dst_points)

        pt1 = [dst_points[i][0], dst_points[i][1]]
        pt2 = [dst_points[next_point_ind][0], dst_points[next_point_ind][1]]

        # Draw circles at varying distances
        delta_x = pt2[0] - pt1[0]
        delta_y = pt2[1] - pt1[1]

        ## Tests
        cir_pt = (int((pt1[0] + (1/3)*delta_x)), int((pt1[1] + (1/3)*delta_y)))

        ### Shift the circle toward the center a bit
        xfc = R - cir_pt[0] # x distance from center
        yfc = R - cir_pt[1] # y distance from center

        shift_factor = 0.10 # Shifts the center point closer to the center by a factor if this much

        shifted_x = int(cir_pt[0] + shift_factor * xfc)
        shifted_y = int(cir_pt[1] + shift_factor * yfc)

        x1, y1, x2, y2 = create_bb(image, (shifted_x, shifted_y - num_offset), sl=side_length)
        subimage = image[y1:y2, x1:x2]
        subimages.append(subimage)

    return subimages

In [6]:
def save_hex_images(image, save_dir, dst_points, side_length, num_offset, ind):
    """ Get all the images for a NN to identify hex types
    dst_points: perimiter points of the board
    side_length: side length of bounding box for each number
    num_offset: the number of pixels above the number point
    """
    subimages = []
    R = 526
    # This is the actual center point
    x1, y1, x2, y2 = create_bb(image,
                               (R, R - num_offset),
                               side_length)
    print(type(x1), type(y1), type(x2), type(y2))
    subimage_center = image[y1:y2, x1:x2]
    subimages.append(subimage_center)
    # This gets all the numbers that fall on the line between the center point of the board and the perimiter points
    for pt in dst_points:
        delta_x = R - pt[0]
        delta_y = R - pt[1]

        x1, y1, x2, y2 = create_bb(image, 
                                   (int((R + (7/24)*delta_x)), 
                                    int((R + (7/24)*delta_y) - num_offset)), 
                                   side_length)
        subimage1 = image[y1:y2, x1:x2]

        x1, y1, x2, y2 = create_bb(image, 
                                   (int((R + (3/5)*delta_x)), 
                                    int((R + (3/5)*delta_y) - num_offset)), 
                                    side_length)
        subimage2 = image[y1:y2, x1:x2]

        subimages.append(subimage1)
        subimages.append(subimage2)

    # This gets all the numbers not found by the center lines to the perimiter points using every other perimiter point to find the numbers
    for i in range(len(dst_points)):
        # Draw lines between every other corner
        next_point_ind = i+2
        if i+2 >= len(dst_points):
            next_point_ind = i+2-len(dst_points)

        pt1 = [dst_points[i][0], dst_points[i][1]]
        pt2 = [dst_points[next_point_ind][0], dst_points[next_point_ind][1]]

        # Draw circles at varying distances
        delta_x = pt2[0] - pt1[0]
        delta_y = pt2[1] - pt1[1]

        ## Tests
        cir_pt = (int((pt1[0] + (1/3)*delta_x)), int((pt1[1] + (1/3)*delta_y)))

        ### Shift the circle toward the center a bit
        xfc = R - cir_pt[0] # x distance from center
        yfc = R - cir_pt[1] # y distance from center

        shift_factor = 0.10 # Shifts the center point closer to the center by a factor if this much

        shifted_x = int(cir_pt[0] + shift_factor * xfc)
        shifted_y = int(cir_pt[1] + shift_factor * yfc)

        x1, y1, x2, y2 = create_bb(image, (shifted_x, shifted_y - num_offset), sl=side_length)
        subimage = image[y1:y2, x1:x2]
        subimages.append(subimage)

    for img in subimages:
        if ind < 10:
            saveImage(f"00{ind}.jpg", img, save_dir)
        elif ind < 100:
            saveImage(f"0{ind}.jpg", img, save_dir)
        else:
            saveImage(f"{ind}.jpg", img, save_dir)
        ind += 1

    return ind

In [10]:
from pathlib import Path
# Get the ground board imgs
board_imgs_dir = Path("../images/v5")
board_imgs = [f for f in board_imgs_dir.iterdir() if f.is_file()]

save_dir = "./ground_imgs"
# hex_imgs = get_hex_images(hom_img, dst_points, side_length=40, num_offset=55)
# for img in hex_imgs:
#     showImage(img)

# Iterate through the board imgs to homography, then save the imgs for future learning
ind = 0
BB_SIDE_LENGTH = 40
PX_OFFSET = 55
print(board_imgs)
for img in board_imgs:
    hom_img, dst_points = homography_board(str(img), vis=False)
    ind = save_hex_images(hom_img, save_dir, dst_points, BB_SIDE_LENGTH, PX_OFFSET, ind)


[PosixPath('../images/v5/board01.jpg'), PosixPath('../images/v5/board00.jpg'), PosixPath('../images/v5/board02.jpg'), PosixPath('../images/v5/board03.jpg'), PosixPath('../images/v5/board07.jpg'), PosixPath('../images/v5/board06.jpg'), PosixPath('../images/v5/board04.jpg'), PosixPath('../images/v5/board05.jpg'), PosixPath('../images/v5/board08.jpg'), PosixPath('../images/v5/board09.jpg')]
<class 'int'> <class 'int'> <class 'int'> <class 'int'>
Image saved to ./ground_imgs/000.jpg
Image saved to ./ground_imgs/001.jpg
Image saved to ./ground_imgs/002.jpg
Image saved to ./ground_imgs/003.jpg
Image saved to ./ground_imgs/004.jpg
Image saved to ./ground_imgs/005.jpg
Image saved to ./ground_imgs/006.jpg
Image saved to ./ground_imgs/007.jpg
Image saved to ./ground_imgs/008.jpg
Image saved to ./ground_imgs/009.jpg
Image saved to ./ground_imgs/010.jpg
Image saved to ./ground_imgs/011.jpg
Image saved to ./ground_imgs/012.jpg
Image saved to ./ground_imgs/013.jpg
Image saved to ./ground_imgs/014.jp